In [9]:
import sys
import os

from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report

from xgboost import XGBClassifier

sys.path.append(os.path.dirname(os.getcwd()))

from dbn_nslkdd.dbn.models import UnsupervisedDBN


In [10]:
DATA_PATH = Path('..') / 'data'
RAW_DATA_PATH = DATA_PATH / 'raw'
PROCESSED_DATA_PATH = DATA_PATH / 'processed_v3'

MODELS_PATH = Path('..') / 'models'

In [14]:
X_train = pd.read_csv(PROCESSED_DATA_PATH / 'X_train.csv')
X_val   = pd.read_csv(PROCESSED_DATA_PATH / 'X_val.csv')
y_train = pd.read_csv(PROCESSED_DATA_PATH / 'y_train.csv')
y_val   = pd.read_csv(PROCESSED_DATA_PATH / 'y_val.csv')
X_test  = pd.read_csv(PROCESSED_DATA_PATH / 'X_test.csv')
y_test  = pd.read_csv(PROCESSED_DATA_PATH / 'y_test.csv')

y_pred_xgboost = np.load(PROCESSED_DATA_PATH / 'y_pred.npy', allow_pickle=True).astype(np.bool_)
y_pred_dbn = np.load(PROCESSED_DATA_PATH / 'dbn_model_results.npy', allow_pickle=True).astype(np.bool_)


combined_models_OR = np.logical_or(y_pred_xgboost, y_pred_dbn)

In [15]:
report = classification_report(y_test.iloc[:22500], combined_models_OR)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      9694
           1       0.89      0.93      0.91     12806

    accuracy                           0.89     22500
   macro avg       0.90      0.89      0.89     22500
weighted avg       0.90      0.89      0.89     22500

